In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import tensorflow as tf

from nn_mobile_net import MobileNet
from keras.preprocessing import image

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from tensorflow.python.tools.inspect_checkpoint import print_tensors_in_checkpoint_file

In [ ]:
print_tensors_in_checkpoint_file('saved_model/mobilenet_v1_1.0_224.ckpt', None, False)

# Load a few images

In [ ]:
local_images = [
    'dog.jpg', 'canoe.jpg', 'car.jpg', 'cat.jpg', 
    'tabby.jpg', 'submarine.jpg', 'snake.jpg', 'teapot.jpg'
]

In [ ]:
pictures = np.zeros((8, 224, 224, 3))
for i, pic in enumerate(local_images):
    x = image.load_img('images/' + pic, target_size=(224, 224))
    pictures[i] = image.img_to_array(x)
    
pictures /= 255.0

# Try it

In [ ]:
model = MobileNet(1001, tf.train.RMSPropOptimizer(1e-3), weight_decay=1e-7)

In [ ]:
graph = model.graph

In [ ]:
with graph.as_default():
    c = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)

In [ ]:
c

In [ ]:
#weights = np.load('')[()]
class_names = np.load('imagenet_class_names.npy')

In [ ]:
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

In [ ]:
X = pictures.copy()
X -= mean
X /= std

In [ ]:
predictions = model.predict_proba(X)

In [ ]:
predicted_classes = class_names[predictions.argsort()[:, 999:] - 1]
predicted_probs = np.sort(predictions)[:, 999:]

In [ ]:
_, axes = plt.subplots(nrows=2, ncols=4, figsize=(17, 8))
axes = axes.flatten()
for i, pic in enumerate(pictures):
    axes[i].set_axis_off();
    axes[i].imshow(pic);
    title = (predicted_classes[i][-1] + ' {0:.3f}' +\
        '\n' + predicted_classes[i][-2] + ' {1:.3f}').\
        format(predicted_probs[i][-1], predicted_probs[i][-2])
    axes[i].set_title(title);
plt.tight_layout()